# Import packages

In [1]:
import os
import time 
import numpy as np
import pandas as pd
import geopandas as gp

In [2]:
from sklearn.linear_model import LinearRegression

In [3]:
import xarray as xr
import rioxarray
from rasterio.enums import Resampling

In [4]:
# Time the script
start_time = time.perf_counter ()

# Set up directories and file paths

In [5]:
path_data_in = '../2 Raw Data'
path_data_out = '../3 Output Data'

## Input data

In [6]:
path_pop_count = os.path.join(path_data_in, 
                              'Population', 
                              'Count',
                              'gha_ppp_2015_1km_Aggregated.tiff')  


dir_pop_agesex= os.path.join(path_data_in, 
                             'Population', 
                             'AgeSex')
#file_healthsiteIO_shp=os.path.join(path_data_in, 
                                 #  'Facilities',
                                  # 'Uzbekistan_HealthsitesIO_SHP',
                                  # "Uzbekistan-node.shp")
#file_healthsiteIO_csv=os.path.join(path_data_out,
                                #   'Uzbekistan_healthsites_io.csv')
path_gdp=os.path.join(path_data_in,
                      'GDP_HDI',
                      'GDP_PPP_30arcsec_v3.nc')
path_nl = os.path.join(path_data_in,
                       'Nighttime_Lights',
                       'gha_viirs_100m_2015.tiff')
#no birth data for Albania
#path_birth=os.path.join(path_data_in,
 #                       'Birth',
  #                      'Uzbekistan_1km_Births',
   #                     'uzb_births_pp_v2_2015.tif')
# path_death = os.path.join(path_data_in,
#                           'Death',
#                           'Albania_death_data_by_age_and_sex_2019.csv') 

## Output data

In [7]:
#path_healthfacility_public = os.path.join(path_data_out,"public_health_facilities.csv")
#path_healthfacility_private = os.path.join(path_data_out,"private_health_facilities.csv")
path_agesex = os.path.join(path_data_out,"age_sex_distribution_1km_2019.csv")
#bank_csv_path = os.path.join(data_in_path,"Egypt_Banks_and_ Atms.csv")

# User-defined functions

In [8]:
def find_grid_index(x,y,longs,lats,x_min=None,x_max=None,y_min=None,y_max=None):
    """
    DESCRIPTION: This function helps to find the index of the longitudes and 
    latitudes in the given longitude/latitude lists that is closest to the 
    given point (x,y) 
    
    INPUT -
    x: latitude of the given point
    y: longtidue of the given point
    longs: reference list of longitudes
    lats: reference list of latitudes
    x_min, x_max: min/max boundaries of longitude
    y_min, y_max: min/max boundaries of latitude
    
    OUTPUT -
    Index of the closest longitude/latitude to point (x,y)
    """
    if x_min is None and x_max is None:
        x_l = longs[0]- (longs[1]-longs[0])
        x_r = longs[-1] + (longs[-1]-longs[-2])
        x_min = min(x_l, x_r)
        x_max = max(x_l, x_r)
    if x < x_min or x > x_max:
        return -1,-1
    
    if y_min is None and y_max is None:
        y_l = lats[0]- (lats[1]-lats[0])
        y_r = lats[-1] + (lats[-1]-lats[-2])
        y_min = min(y_l, y_r)
        y_max = max(y_l, y_r)
    if y < y_min or y > y_max:
        return -1,-1
    
    ncol = np.argmin(abs(longs-x)) 
    nrow = np.argmin(abs(lats-y))
    if nrow<0 or nrow>=(len(lats)-1) or ncol<0 or ncol>=(len(longs)-1):
        return -1, -1
    else:
        return nrow,ncol
def points_in_grid_new(points_df,ref_data,value_colname=None):
    """
    DESCRIPTION: This function finds how many points (and how many non nan points and their sum  
    if a value_colname is provided) lying in the grids centered at the coordinates of the reference data
    
    INPUT -
    points_df: pandas dataframe that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    value_colname: column name for the values in the dataframe
    
    OUTPUT -
    count: How many points are in each grid of the reference coordinate
    non_nan_count: How many non nan points are in each grid of the reference coordinate
    value_sum: The sum of the values of the non nan points within each grid of the reference coordinate
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    non_nan_count = np.zeros(ref_data.values.shape)
    if value_colname is not None:
        value_sum = np.empty(ref_data.values.shape)
        value_sum[:] = np.nan
        values = points_df[value_colname]
    else:
        value_sum = None
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for j, x in enumerate(grid_index):
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
            if value_colname is not None and ~np.isnan(values[j]):
                non_nan_count[x[0],x[1]] = non_nan_count[x[0],x[1]]+1 
                value_sum[x[0],x[1]] = np.nansum([value_sum[x[0],x[1]],values[j]])
    return count, non_nan_count, value_sum
def spatial_aggregate(count_in,value_sum_in,block_size,agg_mode='sum', non_nan_count_in=None):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    agg_mode: aggregation operation mode "sum" or "mean"
    non_nan_count_in: 2D array for the non nan count at each grid, only needed for "mean" mode
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    if agg_mode == 'mean':
        new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
        non_nan_count_in = np.pad(non_nan_count_in,half)
    elif agg_mode == 'sum':
        new_nonnan_count_in = None
    else:
        raise Exception('Error: The function only supports agg_mode for sum or mean.')
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            if agg_mode == 'mean':
                new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    if agg_mode == 'mean':
        new_nonnan_count_in = np.where(new_nonnan_count_in == 0, np.nan, new_nonnan_count_in)
        return_value = return_value/new_nonnan_count_in
        return new_count_in, return_value, new_nonnan_count_in
    else:
        return new_count_in, return_value, new_nonnan_count_in
def layer_aggregate_helper(count_in,value_sum_in,block_size,non_nan_count_in):
    """
    DESCRIPTION: This function apply square sum aggregation with stride 1 for 2D numpy array
    with the same padding
    
    INPUT -
    count_in: 2D numpy array for the count at each grid
    value_sum_in: 2D numpy array for the sum of values at each grid
    block_size: the size of the square sum aggregation
    non_nan_count_in: 2D array for the non nan count at each grid
    
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2

    new_count_in = np.zeros(count_in.shape)
    count_in = np.pad(count_in,half)

    return_value = np.zeros(value_sum_in.shape)
    value_sum_in = np.pad(value_sum_in,half)
    
    new_nonnan_count_in = np.zeros(non_nan_count_in.shape)
    non_nan_count_in = np.pad(non_nan_count_in,half)
  
    for nrow in range(half,count_in.shape[0]-half):
        for ncol in range(half,count_in.shape[1]-half):
            new_count_in[nrow-half,ncol-half] = np.sum(count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            new_nonnan_count_in[nrow-half,ncol-half] = np.sum(non_nan_count_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
            return_value[nrow-half,ncol-half] = np.nansum(value_sum_in[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return_value = np.where(new_nonnan_count_in == 0, np.nan, return_value)
    return new_count_in, return_value, new_nonnan_count_in
def layer_aggregate(value1,count1,value2,count2,mode = 'sum'):
    value1 = np.array(value1)
    value2 = np.array(value2)
    count1 = np.array(count1)
    count2 = np.array(count2)
    value2[np.where(count1==count2)]=value1[np.where(count1==count2)]
    value1 = np.where(np.isnan(value1), 0, value1)
    value2 = np.where(np.isnan(value2), 0, value2)
    value_diff = value2-value1
    count_diff = count2-count1
    if mode == 'sum':
        return_value = value_diff
        return_value = np.where(count_diff==0,np.nan,return_value)
    elif mode == 'mean':
        count_diff = np.where(count_diff == 0, np.nan, count_diff)
        return_value = value_diff/count_diff
    else:
        raise Exception('Only support sum or mean for mode')
    return return_value
def square_helper(arr, y, x, mode):
    """
    DESCRIPTION: This is a helper funtion to do mean or sum in square for 2D numpy array
    
    INPUT - 
    arr: 2D numpy array to be averaged
    (y,x): output dimension
    mode: sum or mean, NAs are ignored 

    OUTPUT - 
    Averaged 2D array
    """
    yy, xx = arr.shape
    if mode == "sum":
        vals = np.nansum(arr.reshape(y, yy//y, x, xx//x),(1,3))
    elif mode == "mean":
        vals = np.nanmean(arr.reshape(y, yy//y, x, xx//x),(1,3))
    else:
        raise Exception("Mode is not suppported, please input `sum' or `mean'")
    return vals
def xarray_square_aggregate(df_raw,stride=10,var_name="band_data",mode="mean"):
    """
    DESCRITPION: This is a function to do square aggregation (mean or sum) for a
    xarray dataArray, generating a new xarray dataArray
    
    INPUT -
    df_raw: 2D xarray DataArray
    stride: Size of the non-overlap square aggregation
    var_name: new DataArray variable name
    mode: square aggregation function
    
    OUTPU -
    A new 2D xarray DataArray with non-overlap square aggregation
    """
    nrow, ncol = df_raw.values.squeeze().shape
    new_nrow,new_ncol = int(nrow/stride), int(ncol/stride)
    data = df_raw.values.squeeze()[0:(new_nrow*stride),0:(new_ncol*stride)]
    new_data = square_helper(data, new_nrow, new_ncol, mode)
    new_x = np.mean(df_raw.coords['x'].values[0:(new_ncol*stride)].reshape(-1,stride),axis=1)
    new_y = np.mean(df_raw.coords['y'].values[0:(new_nrow*stride)].reshape(-1,stride),axis=1)
    # create new datarray
    df = xr.DataArray(
        data=new_data,
        dims=["y", "x"],
        coords=dict(
            x= new_x,
            y= new_y,
        ),
    )
    df.name = var_name
    if df_raw.rio.crs is not None:
        df = df.rio.write_crs(df_raw.rio.crs)
    return df
def grid_reproject(df,ref_data,resample_method = Resampling.nearest):
    """
    DESCRIPTION: This is a function to reproject a xarray dataArray to the coordinates
    of another refrence xarray dataArray.
    
    INPUT - 
    df: The xarray DataArray needing to be reprojected to the new grids
    ref_data: The reference xarray DataArray with reference coordinates (the new grids)
    resample_method: resample method
    
    OUTPUT -
    A new xarray DataArray reprojected to the coordinates of ref_data 
    """
    df = df.rio.reproject_match(ref_data, resampling = Resampling.nearest, nodata= np.nan)
    df = df.assign_coords({
        "x": ref_data.x,
        "y": ref_data.y,
    })
    return df
def points_in_grid(points_df,ref_data):
    """
    DESCRIPTION: This function finds how many points are in the grids centered 
    at the coordinates of the reference data
    
    INPUT -
    points_df: xarray DataArray that needs to be count 
    ref_data: xarray DataArray with reference coordinates
    
    OUTPUT -
    How many points are in each grid of the reference coordinates
    """
    long_list = ref_data.coords['x'].values
    lat_list = ref_data.coords['y'].values
    long_list = np.append(long_list, long_list[-1]+ref_data.rio.resolution()[0])
    lat_list = np.append(lat_list, lat_list[-1]+ref_data.rio.resolution()[1])
    count = np.zeros(ref_data.values.shape)
    grid_index = points_df.apply(lambda row: find_grid_index(row['Long'],row['Lat'],long_list,lat_list), axis=1)
    for x in grid_index:
        if x[0]!=-1 and x[1]!=-1:
            count[x[0],x[1]] += 1
    return count
def aggregate_count(count,block_size):
    """
    DESCRIPTION: This function apply square sum aggregation with sride 1 for 2D numpy array
    with same padding
    
    INPUT -
    count: 2D numpy array
    block_size: the size of the square sum aggregation
    
    OUTPUT - 
    a new 2D numpy array with square sum aggreation in the same size
    """
    if block_size%2 != 1:
        raise Exception('block_size must be an odd integer!')
    half = block_size//2
    new_count = np.zeros(count.shape)
    count = np.pad(count,half)
    for nrow in range(half,count.shape[0]-half):
        for ncol in range(half,count.shape[1]-half):
            new_count[nrow-half,ncol-half] = np.sum(count[(nrow-half):(nrow+half+1),(ncol-half):(ncol+half+1)])
    return new_count
def print_summary(data):
    """
    DESCRIPTION: This is a function to print the sumamry of a xarray DataArray or Dataset
    
    INPUT - 
    data: xarray DataArray or Dataset
    
    OUTPUT - 
    summary statistics of data
    """
    print(f"shape: {data.rio.shape}")
    print(f"resolution: {data.rio.resolution()}")
    print(f"coordinates boundary: {data.rio.bounds()}")
    print(f"CRS: {data.rio.crs}")

# Specify reference coordinates/grids

1. The spatial distribution of population in 2015 South Africa
2. zaf_ppp_2015_1km_Aggregated.tif
3. https://hub.worldpop.org/geodata/summary?id=33887

Here we use the grids of Uzbekistan population data as the reference data.
All other data will be reprojected to this reference grids to make all data consistent in space.

## Population

### Read in population data

In [9]:
pop_raw = rioxarray.open_rasterio(path_pop_count)
display(pop_raw)
print_summary(pop_raw)

<xarray.DataArray (band: 1, y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

shape: (773, 533)
resolution: (0.0083333333, -0.0083333333)
coordinates boundary: (-3.251249987361092, 4.732916852582884, 1.1904166615389087, 11.174583493482885)
CRS: EPSG:4326


## Clean variables and dimensions

### Select band index at 0 

In [10]:
pop = pop_raw.isel(band=0)
pop

<xarray.DataArray (y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
    band         int64 1
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

### Drop extra coordinates dimensions beyond latitude and longtitude


In [11]:
pop = pop.reset_coords(names=['band'],drop=True)
pop.name = 'population'
pop

<xarray.DataArray 'population' (y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

### Make population DataArray as the reference grids

In [12]:
ref_da = pop.copy()
ref_ds = ref_da.to_dataset()
ref_da

<xarray.DataArray 'population' (y: 773, x: 533)>
[412009 values with dtype=float32]
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    STATISTICS_MAXIMUM:  17492.072265625
    STATISTICS_MEAN:     102.50673673986
    STATISTICS_MINIMUM:  0.1342030018568
    STATISTICS_STDDEV:   507.1615317816
    _FillValue:          -99999.0
    scale_factor:        1.0
    add_offset:          0.0

# Data Preprocessing

Process age and gender population data

1. Albania 100m Age and Gender structures in 2020
2.
3. https://www.worldpop.org/geodata/summary?id=50113

We have one Geotiff file for each sex class and each age bin, here we aggregate the 36 Geotiff files into one clean csv file, which will be used to calculate death data at each grid later.

### Set sex bins and age bins 

In [13]:
# sex_bins = ['Female','Male']
# age_bins = ['Age <1 year', 'Age 1 to 4','Age 5 to 9',
#             'Age 10 to 14','Age 15 to 19','Age 20 to 24',
#             'Age 25 to 29','Age 30 to 34','Age 35 to 39',
#             'Age 40 to 44','Age 45 to 49','Age 50 to 54',
#             'Age 55 to 59','Age 60 to 64','Age 65 to 69',
#             'Age 70 to 74','Age 75 to 79','Age 80 plus']
# age_start_bins = [0,1] + list(np.arange(5,80+1,5))

In [14]:
# %%time
# age_sex_df = None
# for sex in sex_bins:
#     for file_no, age in enumerate(age_bins):
#         file_name ="alb_"+sex[0].lower()+"_"+str(age_start_bins[file_no])+"_2020_constrained.tif"
#         col_name = sex + ' ' + age
#         age_sex_raw = xr.open_dataarray(os.path.join(dir_pop_agesex,file_name))
#         age_sex = xarray_square_aggregate(age_sex_raw,stride=10,var_name=col_name, mode="sum")
#         age_sex = grid_reproject(age_sex, pop)
#         df = age_sex.to_dataframe()
#         df = df.drop("spatial_ref",axis=1)
#         age_sex_df = df if age_sex_df is None else age_sex_df.merge(df, left_index=True, right_index=True) 
#         print(col_name)

### Rename and take a look at age and sex distribution

In [15]:
# %%time
# age_sex_df = age_sex_df.reset_index().rename({'x':'Long','y':'Lat'},
#                                              axis=1)
# age_sex_df

In [16]:
# %%time
# age_sex_df.to_csv(path_agesex)

## Process GDP data

1. Gridded global datasets for Gross Domestic Product and Human Development Index over 1990-2015
2. GDP_PPP_30arcsec_v3.nc
3. https://datadryad.org/stash/dataset/doi:10.5061/dryad.dk1j0

### Read in raw gdp data 

In [17]:
gdp_raw = rioxarray.open_rasterio(path_gdp)
display(gdp_raw)
print_summary(gdp_raw)

<xarray.DataArray 'GDP_PPP' (time: 3, y: 21600, x: 43200)>
[2799360000 values with dtype=float32]
Coordinates:
  * time         (time) float64 1.99e+03 2e+03 2.015e+03
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

shape: (21600, 43200)
resolution: (0.008333333380429452, -0.00833333342752775)
coordinates boundary: (-180.00000101727616, -90.00000101729967, 180.00000101727616, 90.0000010172997)
CRS: None


### Clean gdp variables and dimensions

In [18]:
# select year 2015 gdp 
gdp = gdp_raw.isel(time=2)
# drop extra coordinates dimensions beyond latitude and longtitude
gdp = gdp.reset_coords(names=['time'],drop=True)
# specify CRS
gdp = gdp.rio.write_crs("epsg:4326", inplace=True)

gdp

<xarray.DataArray 'GDP_PPP' (y: 21600, x: 43200)>
[933120000 values with dtype=float32]
Coordinates:
  * x            (x) float64 -180.0 -180.0 -180.0 -180.0 ... 180.0 180.0 180.0
  * y            (y) float64 90.0 89.99 89.98 89.97 ... -89.98 -89.99 -90.0
    spatial_ref  int64 0
Attributes:
    long_name:      Gross Domestic Production (GDP) (PPP)
    missing_value:  -9
    units:          ('constant 2011 international US dollar', 'constant 2011 ...
    _FillValue:     -9.0
    scale_factor:   1.0
    add_offset:     0.0

### Reproject gdp to the refrence coordinates

In [19]:
gdp = grid_reproject(gdp, ref_da)
gdp.name = 'GDP_PPP'
gdp

<xarray.DataArray 'GDP_PPP' (y: 773, x: 533)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    long_name:     Gross Domestic Production (GDP) (PPP)
    units:         ('constant 2011 international US dollar', 'constant 2011 i...
    scale_factor:  1.0
    add_offset:    0.0
    _FillValue:    -9.0

## Process Night life lights data

1. VIIRS night-time lights (2012-2016), South Africa
2. zaf_viirs_100m_2015.tiff
3. https://hub.worldpop.org/geodata/summary?id=18738 

### Read in raw nightlight data 

In [20]:
nl_raw = rioxarray.open_rasterio(path_nl)
display(nl_raw)
print_summary(nl_raw)

<xarray.DataArray (band: 1, y: 7721, x: 5337)>
[41206977 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -3.255 -3.254 -3.253 -3.252 ... 1.19 1.191 1.192
  * y            (y) float64 11.17 11.17 11.17 11.17 ... 4.742 4.741 4.74 4.739
    spatial_ref  int64 0
Attributes:
    _FillValue:    3.4028234663852886e+38
    scale_factor:  1.0
    add_offset:    0.0

shape: (7721, 5337)
resolution: (0.00083333333, -0.00083333333)
coordinates boundary: (-3.2554166386500185, 4.738750182420008, 1.1920833435599816, 11.172916823350008)
CRS: EPSG:4326


### Clean nightlight variables and dimensions

In [21]:
# Aggregate the 100m nightlight to 1km with square mean
nl = xarray_square_aggregate(nl_raw,stride=10,
                             var_name="nightlight", 
                             mode="mean")

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


### Reproject the nightlight to reference coordinates

In [22]:
nl = grid_reproject(nl, ref_da)
nl.name = 'nightlight'
display(nl)

<xarray.DataArray 'nightlight' (y: 773, x: 533)>
array([[inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       ...,
       [inf, inf, inf, ..., inf, inf, inf],
       [inf, inf, inf, ..., inf, inf, inf],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Attributes:
    _FillValue:  nan

## Merge GDP, population, nightlight into one dataset

In [23]:
merge_ds = xr.merge([gdp, pop, nl])
display(merge_ds)
print_summary(merge_ds) 

<xarray.Dataset>
Dimensions:      (x: 533, y: 773)
Coordinates:
  * x            (x) float64 -3.247 -3.239 -3.23 -3.222 ... 1.17 1.178 1.186
  * y            (y) float64 11.17 11.16 11.15 11.15 ... 4.762 4.754 4.745 4.737
    spatial_ref  int64 0
Data variables:
    GDP_PPP      (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    population   (y, x) float32 ...
    nightlight   (y, x) float32 inf inf inf inf inf inf ... nan nan nan nan nan
Attributes:
    long_name:     Gross Domestic Production (GDP) (PPP)
    units:         ('constant 2011 international US dollar', 'constant 2011 i...
    scale_factor:  1.0
    add_offset:    0.0
    _FillValue:    -9.0

shape: (773, 533)
resolution: (0.0083333333, -0.0083333333)
coordinates boundary: (-3.251249987361092, 4.732916852582884, 1.1904166615389087, 11.174583493482885)
CRS: EPSG:4326


In [24]:
merge_df = merge_ds.to_dataframe().reset_index()
merge_df

,x,y,spatial_ref,GDP_PPP,population,nightlight
0,-3.247083,11.170417,0,NaN,-99999.0,inf
1,-3.247083,11.162083,0,NaN,-99999.0,inf
2,-3.247083,11.153750,0,NaN,-99999.0,inf
3,-3.247083,11.145417,0,NaN,-99999.0,inf
4,-3.247083,11.137083,0,NaN,-99999.0,inf
...,...,...,...,...,...,...
412004,1.186250,4.770417,0,NaN,-99999.0,inf
412005,1.186250,4.762084,0,NaN,-99999.0,inf
412006,1.186250,4.753750,0,NaN,-99999.0,inf
412007,1.186250,4.745417,0,NaN,-99999.0,inf


In [25]:
merge_df[merge_df['population']<=0]['population'].value_counts()

-99999.0    134008
Name: population, dtype: int64

In [26]:
merge_df.isnull().sum()

x                   0
y                   0
spatial_ref         0
GDP_PPP        347509
population          0
nightlight        533
dtype: int64

In [27]:
# drop observations with missing population or missing GDP [note: missing was coded as -99999]
merge_df2 = (
    merge_df[(merge_df["population"] > 0) & (merge_df["GDP_PPP"].notnull())]
    .copy()
    .reset_index(drop=True)
)

# generate GDP PC
merge_df2["pc"] = merge_df2["GDP_PPP"] / merge_df2["population"]

# rename x and y to longitude and latitude
merge_df2.rename({"x": "Long", "y": "Lat"}, axis="columns", inplace=True)

# drop spatial ref and band
merge_df2.drop(["spatial_ref"], axis="columns", inplace=True)

# take a look at the data
merge_df2


,Long,Lat,GDP_PPP,population,nightlight,pc
0,-3.238750,6.653750,1.739344e+05,189.508072,inf,917.820740
1,-3.230417,6.803750,8.027108e+05,71.738564,inf,11189.390625
2,-3.230417,6.795417,2.033734e+05,64.722328,inf,3142.245361
3,-3.230417,6.787084,1.588169e+05,63.303299,inf,2508.824951
4,-3.230417,6.778750,3.502443e+05,59.484226,inf,5888.019531
...,...,...,...,...,...,...
54013,1.186250,6.145417,6.236881e+05,1289.173218,2.820924,483.789185
54014,1.186250,6.137084,3.040087e+05,1359.127686,2.677600,223.679291
54015,1.186250,6.128750,7.257764e+05,1734.647095,2.861170,418.400024
54016,1.186250,6.120417,7.984543e+06,2745.930908,6.389873,2907.772705


In [36]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
merge_df2.describe()

,Long,Lat,GDP_PPP,population,nightlight,pc
count,54018.00000,54018.00000,54018.00000,54018.00000,54016.00000,54018.00000
mean,-1.29644,6.64576,1969608.25000,331.77429,inf,8597.17969
std,1.02453,1.19396,5959122.50000,1115.38965,NaN,37681.51953
min,-3.23875,4.73708,0.14567,0.28447,0.03288,0.01083
25%,-2.10542,5.82875,72075.88477,62.01600,0.12716,802.12523
50%,-1.41375,6.41208,305610.32812,116.54653,0.19219,2757.85205
75%,-0.55542,7.24542,1335608.00000,247.48012,0.41982,7591.07275
max,1.18625,11.15375,236693696.00000,17492.07227,inf,3609871.75000


### Get percentiles for mapping

GDP percentiles - bottom 40%, 40-80%, 80-95%, above 95%

In [29]:
gdp_values = merge_df2['GDP_PPP']
gdp_cutoff = np.nanquantile(gdp_values,[0.4, 0.8, 0.95, 1])

# turn off scientific notation in numpy arrays
np.set_printoptions(formatter={'float': '{:0.4f}'.format})
print(gdp_cutoff)

[177980.6406 1892142.2250 8851016.7500 236693696.0000]


GDP PC percentiles - bottom 40%, 40-80%, 80-95%, above 95%

In [37]:
gdp_pc_values = merge_df2['pc']
gdp_pc_cutoff = np.nanquantile(gdp_pc_values,[0.4, 0.8, 0.95, 1])

# turn off scientific notation in numpy arrays
np.set_printoptions(formatter={'float': '{:0.4f}'.format})
print(gdp_pc_cutoff)

[1768.5456 9631.7695 28869.2498 3609871.7500]


Population percentiles - bottom 10%, 10-50%, 50-75%, 75-90%, 90-95%, 95-99%, above 99%

In [31]:
pop_values = merge_df2['population']
pop_cutoff = np.nanquantile(pop_values,[0.1, 0.5, 0.75, 0.9, 0.95, 0.99, 1])
pop_cutoff

array([37.1396, 116.5465, 247.4801, 505.4928, 922.3072, 4641.0271,
       17492.0723])

# Data Analysis

## GDP rating

In [32]:
# # assign gdp rating 
# gdp_values = merge_df2['GDP_PPP']
# gdp_cutoff = np.nanquantile(gdp_values,[0.7,0.95])
# gdp_levels = np.array(['nan']*len(gdp_values))
# gdp_levels[~np.isnan(gdp_values)] = 'C'
# gdp_levels[gdp_values > gdp_cutoff[0]] = 'B'
# gdp_levels[gdp_values > gdp_cutoff[1]] = 'A'
# merge_df2['gdp_rating'] = gdp_levels
# merge_df2['gdp_rating'].value_counts()

In [33]:
# %%time
# merge_df2 = merge_df2.rename({'x':'Long','y':'Lat'},axis=1)
# if read in age sex csv, there will be some decimal rounding difference due to float type
# merge_df2['Lat_5_decimal'] = np.round(merge_df2['Lat'],5).astype('str')
# merge_df2['Long_5_decimal'] = np.round(merge_df2['Long'],5).astype('str')
# display(merge_df2.head())


# Save the final data

In [34]:
%%time
merge_df2.to_csv(os.path.join(path_data_out, "gha_final_data_v0.csv"))

CPU times: user 314 ms, sys: 30.3 ms, total: 345 ms
Wall time: 349 ms


# Total runtime

In [35]:
end_time = time.perf_counter ()
print(round((end_time - start_time)/60,2), "minutes")

0.4 minutes
